In [ ]:
import suite2p
from tempfile import TemporaryDirectory

### set parameters

In [ ]:
ops = suite2p.default_ops()
print(ops)

### set data path

In [ ]:
db = {
    'data_path': ['../data/'],
    'save_path0': TemporaryDirectory().name,
    'tiff_list': ['Sue_2x_3000_40_-46.tif'],
}
db

### run suite2p

In [ ]:
output_ops = suite2p.run_s2p(ops=ops, db=db)

### result

In [ ]:
from pathlib import Path

output_op = output_ops
print('param:\n', set(output_op.keys()).difference(ops.keys()))
print('\noutput path: \n', list(Path(output_op['save_path']).iterdir()))

### motion correction (rigid)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 4))
plt.subplot(1, 4, 1)
plt.imshow(output_op['refImg'], cmap='gray', )
plt.title("Reference Image for Registration");

plt.subplot(1, 4, 2)
plt.imshow(output_op['max_proj'], cmap='gray')
plt.title("Registered Image, Max Projection");

plt.subplot(1, 4, 3)
plt.imshow(output_op['meanImg'], cmap='gray')
plt.title("Mean registered image")

plt.subplot(1, 4, 4)
plt.imshow(output_op['meanImgE'], cmap='gray')
plt.title("High-pass filtered Mean registered image");

### load result

In [ ]:
print('save path: ', output_ops['save_path0'])

In [ ]:
output_ops

In [ ]:
import os
import numpy as np

path = os.path.join('../logs', 'suite2p', 'plane0', 'ops.npy')

assert os.path.exists(path)

load_data = np.load(path, allow_pickle=True)
load_data = load_data.item()

In [ ]:
plt.imshow(output_op['refImg'], cmap='gray', )

### detection

In [ ]:
from suite2p import ROI

stats_file = Path(output_op['save_path']).joinpath('stat.npy')
iscell = np.load(Path(output_op['save_path']).joinpath('iscell.npy'), allow_pickle=True)[:, 0].astype(bool)
stats = np.load(stats_file, allow_pickle=True)
stats.shape, iscell.shape

In [ ]:
label_id = True
arrays = []
for i, s in enumerate(stats):
    array = ROI(ypix=s['ypix'], xpix=s['xpix'], lam=s['lam'], med=s['med'], do_crop=False).to_array(Ly=output_op['Ly'], Lx=output_op['Lx'])
    if label_id:
        array *= i + 1
    arrays.append(array)
im = np.stack(arrays)
im[im == 0] = np.nan

In [ ]:
plt.figure(figsize=(16, 4))

plt.subplot(1, 4, 1)
plt.imshow(output_op['max_proj'], cmap='gray')
plt.title("Registered Image, Max Projection")

plt.subplot(1, 4, 2)
plt.imshow(np.nanmax(im, axis=0), cmap='jet')
plt.title("All ROIs Found")

plt.subplot(1, 4, 3)
plt.imshow(np.nanmax(im[~iscell], axis=0, ), cmap='jet')
plt.title("All Non-Cell ROIs")

plt.subplot(1, 4, 4)
plt.imshow(np.nanmax(im[iscell], axis=0), cmap='jet')
plt.title("All Cell ROIs");

### Extract trace

In [ ]:
f_cells = np.load(Path(output_op['save_path']).joinpath('F.npy'))
f_neuropils = np.load(Path(output_op['save_path']).joinpath('Fneu.npy'))
spks = np.load(Path(output_op['save_path']).joinpath('spks.npy'))
f_cells.shape, f_neuropils.shape, spks.shape

In [ ]:
plt.figure(figsize=[20,60])
plt.suptitle("Flourescence and Deconvolved Traces for Different ROIs", y=0.92);
rois = np.arange(len(f_cells))[::20]
for i, roi in enumerate(rois):
    plt.subplot(len(rois), 1, i+1, )
    f = f_cells[roi]
    f_neu = f_neuropils[roi]
    sp = spks[roi]
    # Adjust spks range to match range of fluroescence traces
    fmax = np.maximum(f.max(), f_neu.max())
    fmin = np.minimum(f.min(), f_neu.min())
    frange = fmax - fmin 
    sp /= sp.max()
    sp *= frange
    plt.plot(f, label="Cell Fluorescence")
    plt.plot(f_neu, label="Neuropil Fluorescence")
    plt.plot(sp + fmin, label="Deconvolved")
    plt.xticks(np.arange(0, f_cells.shape[1], f_cells.shape[1]/10))
    plt.ylabel(f"ROI {roi}", rotation=0)
    plt.xlabel("frame")
    if i == 0:
        plt.legend(bbox_to_anchor=(0.93, 2))

### spike deconvolution

In [ ]:
# compute deconvolution
from suite2p.extraction import dcnv
import numpy as np

tau = 1.0 # timescale of indicator
fs = 30.0 # sampling rate in Hz
neucoeff = 0.7 # neuropil coefficient
# for computing and subtracting baseline
baseline = 'maximin' # take the running max of the running min after smoothing with␣

# gaussian
sig_baseline = 10.0 # in bins, standard deviation of gaussian with which to smooth
win_baseline = 60.0 # in seconds, window in which to compute max/min filters
prctile_baseline = 8
batch_size = 200
ops = {'tau': tau, 'fs': fs, 'neucoeff': neucoeff, 'baseline': baseline, 'sig_baseline': sig_baseline, 
       'win_baseline': win_baseline, 'prctile_baseline': prctile_baseline, 'batch_size': batch_size}

# load traces and subtract neuropil
F = np.load(os.path.join('../logs', 'suite2p', 'plane0', 'F.npy'))
Fneu = np.load(os.path.join('../logs', 'suite2p', 'plane0', 'Fneu.npy'))#np.load('Fneu.npy')
Fc = F - ops['neucoeff'] * Fneu

# baseline operation
Fc = dcnv.preprocess(
    F=Fc,
    baseline=ops['baseline'],
    win_baseline=ops['win_baseline'],
    sig_baseline=ops['sig_baseline'],
    fs=ops['fs'],
    prctile_baseline=ops['prctile_baseline']
)

# get spikes
spks = dcnv.oasis(F=Fc, batch_size=ops['batch_size'], tau=ops['tau'], fs=ops['fs'])

In [ ]:
plt.plot(spks.flatten())

In [ ]:
output_ops

In [ ]:
plt.figure(figsize=(16, 4))
plt.imshow((spks*255).astype(np.uint8))

In [ ]:
(spks*255).astype(np.uint8).shape

### Registration(motion correction)

In [ ]:
ops = suite2p.default_ops()

In [ ]:
output_ops